### TMA4215 Numerisk Matematikk 

Autumn 2021 – August 27, 2021

Problems created and reviewed by: R. Bergmann, E. Çokaj, O. P. Hellan 

Problems answered by: I. M. Sandum

# Problem Sheet 1

## Deadline
September 5, 2021, 23:59


## Submission
submit your Jupyter notebook containing the solution via upload in blackboard.

## Problem 1.

Let $A\in \mathbb R^{n\times n}$ be an invertible matrix and $b\in\mathbb R^n$ be given.
    Consider the problem of solving the linear system of equations “Find $x\in \mathbb R^n$ such that $Ax=b$”.

1. Suppose we find $x$ with an error $\delta x$ due to an error in $b$ of size $\delta b$. This means we actually solved
   $$ A(x+\delta x) = b + \delta b$$
   Derive upper bounds for the absolute condition number $\frac{\lVert \delta x \rVert}{\lVert\delta b\rVert} \leq K_{\mathrm{abs}}(b)$
   and the relative condition number $\frac{\lVert\delta x \rVert / \lVert x\rVert}{\lVert \delta b\rVert / \lVert b \rVert } \leq K(b)$ for the case of the $2$-norm in terms of the Eigenvalues of $A$.

   Main ingredients are the $2$-norm of a vector, the (compatible, induced) $2$-norm (or spectral norm) of a matrix and their compatibility, i.e. that $\lVert Ax\rVert_2 \leq \lVert A\rVert_2 \lVert x\rVert_2$ holds.


2. Let further $A$ be disturbed data, i. e. we have a disturbed solution $x+\delta x$ often
   $$
   (A+\delta A)(x+\delta x) = b+\delta b
   $$
   How does the relative error $\frac{\lVert \delta x \rVert}{\lVert x \rVert}$ depend on the two relative errors $\frac{\lVert \delta b \rVert}{\lVert b \rVert}$ and $\frac{\lVert \delta A \rVert}{\lVert a \rVert}$ of the data?

3. What can you say **a priori** (before you solve any system) about relative errors when
$$
  Ax = \begin{pmatrix}
  1 & 1\\
  1.0004 & 1
  \end{pmatrix}x
  = \begin{pmatrix}
    2\\2
  \end{pmatrix} = b
$$
  is given (you can use Python to reason for your answer)?
  Compute the exact solution by hand and use `Python` to solve the disturbed version with $\delta b = \begin{pmatrix} 2\\2.001 \end{pmatrix}$

1. eq. 2.7 in the book gives $K_{rel}(b)\approx ||G'(b)|| \frac{||b||}{||G(b)||} = \frac{||A^{-1}||\cdot||b||}{||A^{-1}b||}=\frac{||A^{-1}||\cdot||Ax||}{||x||}\leq ||A^{-1}||\cdot||A|| = K(A)$ (eq. 3.4), where the less than or equal part comes from eq 1.19. Furthermore, eq. 3.5 gives $K_2(A)=\frac{\lambda_{max}}{\lambda_{min}}$

    2.7 also gives $K_{abs}(b)\approx||G'(b)||=||A^{-1}||=\frac{K_2(A)}{||A_2||}=\frac{\lambda_{max}/\lambda_{min}}{\lambda_{max}}=\frac{1}{\lambda_{min}}$
    
2. According to eq. 3.9 in the book, we have that $\frac{||\delta x||}{||x||}\leq\frac{K(A)}{1-K(A)||\delta A||/||A||}(\frac{||\delta b||}{||b||}+\frac{||\delta A||}{||A||})$, so the two relative errors of the data creates an upper bound of sorts.


3. If we calculate the condition number, we can know in advance if the relative error is going to be big or small. For example, the condition number for this matrix A (calculated below) is $\approx 10002$, which is very high. I can therefore expect a big error for the disturbed version. From the calculations below I can confirm that the perturbed solution is quite wrong. 

In [21]:
import numpy as np
A=np.matrix([[1,1],[1.0004,1]])
print("Condition number for A: ", np.linalg.cond(A))

b=np.array([2,2])
print("The exact solution is " , np.linalg.solve(A,b))

db=np.array([2,2.001])
diffx=np.array([0,0.001])
print("The perturbed solution is ", np.linalg.solve(A,db))
#print(np.linalg.norm(np.linalg.solve(A,b))/np.linalg.norm(np.linalg.solve(A,db)))

Condition number for A:  10002.000300008634
The exact solution is  [0. 2.]
The perturbed solution is  [ 2.5 -0.5]


## Problem 2.

We want to compute the LU factorisation with pivoting of a matrix $A$

$$
     PA = LU
$$

where $P$ is a permutation matrix, $L$ is a lower-triangular matrix with unit diagonal, and $U$ is an upper-triangular matrix.
We represent the matrices in question as follows:
The permutation matrix $P$ is $n\times n$, but is represented as a vector $\mathtt{P}$ such that row number $k$ in $P$ is the canonical unit vector $e_{\mathtt{P}_k}$. Let us illustrate this by an example

$$
\mathtt{P}=
\left[
\begin{array}{r} 3 \\ 1 \\ 2 \end{array}
\right]\quad\Rightarrow\quad
P=\left[
\begin{array}{ccc}
0 & 0 & 1 \\ 1 & 0 & 0 \\ 0 & 1 & 0
\end{array}
\right]
$$

We stipulate that a Python function takes a two-dimensional numpy-array $\mathtt{A}$ as input, and returns
an *over-written* $\mathtt{A}$ which contains $L$ and $U$ in the following sense upon return:
x
$$
\begin{array}{ll}
\mathtt{A}[\mathtt{P}[i],j] = L_{ij} & \text{for}\ i<j \\
\mathtt{A}[\mathtt{P}[i],j] = U_{ij} & \text{for}\ i\geq j
\end{array}
$$

That $L$ has 1 on the diagonal is always the case, so the diagonal of $L$ needs not be stored. The remaining elements of $L$ and $U$ are zero and need not be stored either. The algorithm can be formulated as follows (compare to text book):

- Input: $A$ of size $n\times n$
- Initialisation
    * Let $P_i = i,\ i=0,\ldots,n-1$ be a vector (array) with $n$ components
- for $k$ **in** range(n-1):
    1. Find index $P_\ell$ such that $|\mathtt{A}_{P_\ell,k}|=\max_{k\leq i \leq n-1} |\mathtt{A}_{P_i,k}|$, i.e. scan column $k$ from the diagonal and down for the largest element in absolute value. 
    2. Swap $P_k$ by $P_\ell$.
    3. Find multipliers $A_{P_i,k}\leftarrow \frac{A_{P_i,k}}{A_{P_k,k}},\ i=k+1,\ldots,n-1$.
    4. Perform elimination, i.e. $A_{P_i,j}\leftarrow A_{P_i,j}-A_{P_i,k}\cdot A_{P_k,j},\ i,j=k+1,\ldots,n-1$
- Output: A,P

There are – of course – implementations of these, often highly optimised for special cases (e.g. when $A$ is sparse) but here we first want to learn how to code it ourselves. Let's also use this to our advantage


1. Write a function for LU-factorisation with row-wise pivoting as indicated above.
   A template could be

       def mylu(A):
   
    
   and it should return the pivot vector (permutation vector) $\mathtt{P}$, and over-written  version of $A$. You can also choose to copy $A$ into some other matrix $\mathtt{LU}$ from the beginning using e.g. 

       LU = A.copy()

2. use the function `scipy.linalg.lu` to test your implementation from the first part. Write a test function

       def mylutest(A)
   
   that compares the result of your implementation to the one from `SciPy`. Call this function for example with

   $$
   A = \begin{pmatrix} 2 & 5 & 8 & 7\\ 5 & 2 & 2 & 8 \\ 7 & 5 & 6 & 6\\ 5 & 4 & 4 & 8\end{pmatrix}
   $$
   
   (or `np.array([[2, 5, 8, 7], [5, 2, 2, 8], [7, 5, 6, 6], [5, 4, 4, 8]])` to easier copy it).
   
3. Test your function with a matrix `A` that does not meet our assumption of having full rank, for example by repeating a column. What happens?

In [287]:
from scipy.linalg import lu

def myLu(A): #A nxn array
    A_new = np.copy(A)
    P=np.arange(A.shape[0],dtype=int)
    
    for k in range(0,A.shape[0]-1):  #every column 
        M = -1 #need this one to reset for every column 
        ind_row = -1 #same, index row
        for i in range(k,A.shape[0]): #every row from diagonal and down
            if (abs(A[P[i],k]) > M):
                M = abs(A[P[i],k])
                ind_row = i
        temp = P[k]
        P[k] = P[ind_row]
        P[ind_row] = temp
        for i in range(k+1,A.shape[0]):
            A_new[P[i],k]=A_new[P[i],k]/A_new[P[k],k]
        for i in range(k+1,A.shape[0]):
            for j in range(k+1, A.shape[0]):
                A_new[P[i],j]=A_new[P[i],j]-A_new[P[i],k]*A_new[P[k],j]
        
    return A_new,P

#This function uses the permutation vector to give the returned A from myLu(A)
#the right order. 
def PA(P,A):
    a=np.copy(A)
    n=a.shape[0]
    for i in range(n):
        for j in range(n):
            a[i,j]=A[P[i],j]
    return a

In [510]:
def myLuTest(A):
    A_lu , P_lu = myLu(A)
    a=PA(P_lu,A_lu)
    lu,p =scipy.linalg.lu_factor(A)
    for i in range(A.shape[0]):    
        for j in range(A.shape[0]):
            if(round(float(a[i,j]),5)!= round(float(lu[i,j]),5)):
                print("The matrices are not the same...\n")
                return
    print("The matrices are the same, myLu works as it should.\n")
              

In [512]:
test = np.array([[2, 5, 8, 7], [5, 2, 2, 8], [7, 5, 6, 6], [5, 4, 4, 8],],dtype=float)
test2=np.array([[3,2,1,4],[5,3,2,1],[3,2,1,4],[1,1,2,3]])

print("Testing with full rank:")
myLuTest(test)

print("Testing with a repeated column: ")
myLuTest(test2)

print("It seems as if myLu does not work for matrices without full rank")

Testing with full rank:
The matrices are the same, myLu works as it should.

Testing with a repeated column: 
The matrices are not the same...

It seems as if myLu does not work for matrices without full rank


C:\Users\Bruker\Documents\anaconda32\lib\site-packages\ipykernel_launcher.py:4: LinAlgWarning: Diagonal number 4 is exactly zero. Singular matrix.
  after removing the cwd from sys.path.


## Problem 3

1. Implement a

  `def forward_substitution(A,b):`
    
  function that takes a lower triangular matrix `A` and some vector `b` to solve $A\mathbf{x} = \mathbf{b}$

2. Implement a

  `def backward_substitution(A,b):` 
        
  function that takes an upper triangualr matrix `A` and some vector `b` to solve $A\mathbf{x} = \mathbf{b}$ for this case.
  
3. Combine the last two parts with Problem 2 and implement a function

   `def my_solve(A,b):`

   for a square matrix `A` and a right hand side `b` that computes the LU decomposition of `A` and then uses the first two parts of this problem to compute the solution to $A\mathbf{x} = \mathbf{b}$
   
4. Performance. Let's compare our implementation for two cases as well as to the original. To be precise
  * fix some $n$, say `n=100`
  * generate a reguar square matrix `A` (Hint: maybe create `L` and `U` here to be sure `A` is regular)
  * generate `m`, say `m=200` right hand sides `b_k`
  * run an experiment where you time

      a) calling `m` times `my_solve(A,b)` once for each `b_k`

      b) only computing the LU decomposition once and use backward and forward substitutions `m` times

      c) using `np.linalg.solve`
      
    Where do the time differences from a) to b) come from and where the ones from c) to b)? 

In [316]:
def forwardSubstitution(A,b): #A lower triangular
    n=A.shape[0]
    x=np.zeros(n) #solution vector
    x[0]=b[0]/A[0,0]
    for i in range(1,n):
        sum=0
        for j in range(0,i):
            sum += (A[i,j]*x[j])
        x[i]=1/A[i,i] * (b[i]-sum)
    return x

In [317]:
def backwardSubstitution(A,b): #A upper triangular
    n=A.shape[0]
    x=np.zeros(n)
    x[n-1]=b[n-1]/A[n-1,n-1]
    for i in range(n-1,-1,-1):
        sum = 0
        for j in range(i+1,n):
            sum += (A[i,j]*x[j])
            x[i]=1/A[i,i] * (b[i]-sum)
    return x

In [471]:
def my_solve(A,b):
    A_lu,P_lu=myLu(A)
    a=PA(P_lu,A_lu) #rearrange A_lu
    l=np.copy(A)
    u=np.copy(A)
    n=A.shape[0]
    for i in range(n):
        for j in range(n):
            if i==j:
                l[i,j]=1
                u[i,j]=a[i,j]
            elif i<j:    #Opposite of what it says, but this gives the right matrices (?)
                l[i,j] = 0
                u[i,j] = a[i,j]
            else: 
                l[i,j]=a[i,j]
                u[i,j]=0
    y=forwardSubstitution(l,b[P_lu]) #b needs to be pivoted too
    x=backwardSubstitution(u,y)
    return x
    

In [491]:
b=np.array([1,0,3,2])
x= my_solve(test,b)
print(x)
xrett=scipy.linalg.solve(test,b)
print(xrett)

[ 0.06185567  1.70103093 -0.70103093 -0.28865979]
[ 0.06185567  1.70103093 -0.70103093 -0.28865979]


In [505]:
import random
import time
random.seed(time.time())

n=100
m=1000
matrix = np.array([[4,2,5,1],[1,5,6,6],[9,3,3,4],[1,1,1,4]])
b=np.arange(matrix.shape[0])

def random_b():
    for i in range(matrix.shape[0]):
        b[i]=random.randint(1,10)
    return b
def my_solve_m(m,matrix):
    for i in range(m):
        b=random_b()
        x=my_solve(matrix,b)

def my_solve2(n,l,u,P_lu):
    b=random_b()
    y=forwardSubstitution(l,b[P_lu]) 
    x=backwardSubstitution(u,y)
    return x

def my_solve2_m(m,matrix):
    A_lu,P_lu=myLu(matrix)
    a=PA(P_lu,A_lu) 
    l=np.copy(matrix)
    u=np.copy(matrix)
    n=A.shape[0]
    for i in range(n):
        for j in range(n):
            if i==j:
                l[i,j]=1
                u[i,j]=a[i,j]
            elif i<j:    
                l[i,j] = 0
                u[i,j] = a[i,j]
            else: 
                l[i,j]=a[i,j]
                u[i,j]=0
    for i in range(m):
        b=random_b()
        x=my_solve2(n,l,u,P_lu)
        
def linalg_solve_m(m,matrix):
    for i in range(m):
        b=random_b()
        x=scipy.linalg.solve(matrix,b)

print("a) my_solve m times:")        
start_time1 = time.time()
my_solve_m(m,matrix)
print("--- %s seconds ---" % (time.time() - start_time1))

print("b) my solve m times, but LU only once:")
start_time2 = time.time()
my_solve2_m(m,matrix)
print("--- %s seconds ---" % (time.time() - start_time2))

print("c) Linalg_solve m times:")
start_time3 = time.time()
linalg_solve_m(m,matrix)
print("--- %s seconds ---" % (time.time() - start_time3))

print("______________________")
print("The time difference from a to b comes from not having to compute the LU-factorization more than once. ")
print("The time difference from b to c is much smaller, I suppose linalg.solve also only compute LU-factorization once,")
print("and also have much better optimalisation in the code.")

a) my_solve m times:
--- 0.26519179344177246 seconds ---
b) my solve m times, but LU only once:
--- 0.10348391532897949 seconds ---
c) Linalg_solve m times:
--- 0.07019948959350586 seconds ---
______________________
The time difference from a to b comes from not having to compute the LU-factorization more than once. 
The time difference from b to c is much smaller, I suppose linalg.solve also only compute LU-factorization once,
and also have much better optimalisation in the code.
